In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

def optimize_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif df[col].dtype == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

def mainBareBones():
    # Load dataset
    print("Loading dataset...")
    try:
        df = pd.read_csv('Active_Wiretap_dataset.csv', header=None, low_memory=False)
        labels = pd.read_csv('Active_Wiretap_labels.csv', names=['label'], header=None, low_memory=False)

        min_rows = min(len(df), len(labels))
        df = df.iloc[:min_rows]
        labels = labels.iloc[:min_rows]
        labels['label'] = pd.to_numeric(labels['label'], errors='coerce')

        print(f"Dataset shape: {df.shape}")
        print(f"Labels shape: {labels.shape}")
        print(f"Label values found: {labels['label'].unique()}")
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        exit()

    # Preprocess and clean data
    print("\nPreprocessing data...")
    df.columns = [f"feature_{i}" for i in range(df.shape[1])]
    df['label'] = labels['label']
    df = optimize_dtypes(df)
    df = df.dropna()
    df = df[df['label'].notna()]
    print(f"Shape after cleaning: {df.shape}")

    # Split features and labels
    X = df.drop('label', axis=1)
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Feature selection
    print("Performing feature selection...")
    selector = SelectKBest(score_func=f_classif, k=min(30, X.shape[1]))
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    # Train model
    print("\nTraining the model...")
    model = RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1,
        max_depth=10,
        class_weight='balanced'
    )
    model.fit(X_train_selected, y_train)

    # Predictions and evaluation
    print("Making predictions...")
    y_pred = model.predict(X_test_selected)

    print("\nModel Evaluation:")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    print("\nProcess completed!")
    print(f"Total samples: {len(df)}")
    print(f"Number of features: {X.shape[1]}")
    print(f"Number of selected features: {X_train_selected.shape[1]}")
    print(f"Label distribution:\n{y.value_counts()}")

if __name__ == "__main__":
    mainBareBones()


In [ ]:
import pandas as pd
import cupy as cp
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

def optimize_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif df[col].dtype == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

def maincuPY():
    # Load dataset
    print("Loading dataset...")
    try:
        df = pd.read_csv('Active_Wiretap_dataset.csv', header=None, low_memory=False)
        labels = pd.read_csv('Active_Wiretap_labels.csv', names=['label'], header=None, low_memory=False)

        # Ensure datasets have the same number of rows
        min_rows = min(len(df), len(labels))
        df = df.iloc[:min_rows]
        labels = labels.iloc[:min_rows]
        labels['label'] = pd.to_numeric(labels['label'], errors='coerce')

        print(f"Dataset shape: {df.shape}")
        print(f"Labels shape: {labels.shape}")
        print(f"Label values found: {labels['label'].unique()}")
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        exit()

    # Preprocess and clean data
    print("\nPreprocessing data...")
    df.columns = [f"feature_{i}" for i in range(df.shape[1])]
    df['label'] = labels['label']
    df = optimize_dtypes(df)
    df = df.dropna()
    df = df[df['label'].notna()]
    print(f"Shape after cleaning: {df.shape}")

    # Split features and labels
    X = df.drop('label', axis=1).to_numpy(dtype=cp.float32)
    y = df['label'].to_numpy(dtype=cp.float32)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Feature selection
    print("Performing feature selection...")
    X_train_cpu = cp.asnumpy(X_train)  # Convert to NumPy for sklearn
    y_train_cpu = cp.asnumpy(y_train)
    selector = SelectKBest(score_func=f_classif, k=min(30, X.shape[1]))
    X_train_selected_cpu = selector.fit_transform(X_train_cpu, y_train_cpu)
    X_test_selected_cpu = selector.transform(cp.asnumpy(X_test))

    # Convert back to CuPy
    X_train_selected = cp.array(X_train_selected_cpu, dtype=cp.float32)
    X_test_selected = cp.array(X_test_selected_cpu, dtype=cp.float32)

    # Train model
    print("\nTraining the model...")
    model = RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1,
        max_depth=10,
        class_weight='balanced'
    )
    model.fit(cp.asnumpy(X_train_selected), cp.asnumpy(y_train))  # Convert CuPy to NumPy for sklearn

    # Predictions and evaluation
    print("Making predictions...")
    y_pred = model.predict(cp.asnumpy(X_test_selected))  # Predictions on NumPy arrays

    # Evaluate model
    print("\nModel Evaluation:")
    print("\nClassification Report:")
    print(classification_report(cp.asnumpy(y_test), y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(cp.asnumpy(y_test), y_pred))

    # Summary information
    print("\nProcess completed!")
    print(f"Total samples: {len(df)}")
    print(f"Number of features: {X.shape[1]}")
    print(f"Number of selected features: {X_train_selected.shape[1]}")
    print(f"Label distribution:\n{cp.bincount(cp.array(y, dtype=cp.int32))}")

if __name__ == "__main__":
    maincuPY()


In [ ]:
from sklearnex import patch_sklearn  
patch_sklearn()  # Apply Scikit-learn algorithms
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
warnings.filterwarnings('ignore', category=FutureWarning)

def optimize_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif df[col].dtype == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

def mainIntelex():
    # Load dataset
    print("Loading dataset...")
    try:
        df = pd.read_csv('Active_Wiretap_dataset.csv', header=None, low_memory=False)
        labels = pd.read_csv('Active_Wiretap_labels.csv', names=['label'], header=None, low_memory=False)

        min_rows = min(len(df), len(labels))
        df = df.iloc[:min_rows]
        labels = labels.iloc[:min_rows]
        labels['label'] = pd.to_numeric(labels['label'], errors='coerce')

        print(f"Dataset shape: {df.shape}")
        print(f"Labels shape: {labels.shape}")
        print(f"Label values found: {labels['label'].unique()}")
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        exit()

    # Preprocess and clean data
    print("\nPreprocessing data...")
    df.columns = [f"feature_{i}" for i in range(df.shape[1])]
    df['label'] = labels['label']
    df = optimize_dtypes(df)
    df = df.dropna()
    df = df[df['label'].notna()]
    print(f"Shape after cleaning: {df.shape}")

    # Split features and labels
    X = df.drop('label', axis=1)
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Feature selection
    print("Performing feature selection...")
    selector = SelectKBest(score_func=f_classif, k=min(30, X.shape[1]))
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    # Train model
    print("\nTraining the model...")
    model = RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1,
        max_depth=10,
        class_weight='balanced'
    )
    model.fit(X_train_selected, y_train)

    # Predictions and evaluation
    print("Making predictions...")
    y_pred = model.predict(X_test_selected)

    print("\nModel Evaluation:")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    print("\nProcess completed!")
    print(f"Total samples: {len(df)}")
    print(f"Number of features: {X.shape[1]}")
    print(f"Number of selected features: {X_train_selected.shape[1]}")
    print(f"Label distribution:\n{y.value_counts()}")

if __name__ == "__main__":
    mainIntelex()
